# <font color='navy'> <b> Vytvorenie simulačného modelu pomocou knižnice lib2d </b></font>

Pre tvorbu simulačnýchch modelov v 2D prilárne pre oblasť elektrostatiky bola vytvorená jednoduchá knižnica obsahujúca elementárne entity (Rectangle, Circle ...). Knižnica umožňuje kombináciu entít a tvorbu oblastí s orientovanými hranicami. 

Knižnica je objektovo-orientovaná, pozostáva z hlavnej triedy Shape, ktorá obsahuje dátové štruktúry spoločné pre celý model a agreguje triedy reprezentujúce jednotlivé enity. Manipulácia s objektami a vnútorná štruktúra objektov kopíruje postupy a štruktúry skriptovacieho jazyka gmsh.

Pre jednoduchšiu syntax a zjednodušenie vytvárania elementov modelov sú nad knižnicou definované shadow funkcie, tieto prekrývajú klasickú syntax jazyka Python potrebnú pre prácu s objektovými štruktúrami. Pri tvorbe modelu je možné využívať objektovú štruktúru knižnice ako aj všetky funkcie a metódy implementované v gmsh-api pre Python.  

    1D Objekty
    	Point
    	Line
    	PolyLine
    	Arc
    	Spline

    2D Objekty
    	Rectagle
    	RectangleRound
    	RectangleBevel
    	Polygon
    	Circle
    	CircleArc


Pre vizualizáciu hraníc oblastí a orientáciu segmentov je použitá knižniva *CircuitMacros*.


In [1]:
%reset -f --aggressive

from utils.lib2d import *

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)


R = Rectangle( 5, 5, 10, 10, 'Rectangle')
C = Circle(3,3,2, 'Circle')
S = PlaneSurface([R,C])          # plocha R s dierami C1,C2



P1  = Point(R.nw.x, (R.nw.y + R.sw.y)/2)    # bod v polovici usecky R1.n 
Ls  = Model.splitLine(R.w, P1)  

PhysicalSurface(S, "Plocha") 
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

#gmsh.write("./data/lib2d_02.vtk")
#gmsh.write("./data/lib2d_02.msh")
#gmsh.write("./data/lib2d_02.geo_unrolled")

#gmsh.fltk.run()
gmsh.finalize()



s1 = Plot(R)
s1 = s1 + Plot(C)
#print(s1)
from cm.utils import *
cm_compile('obr', s1, grid=True, dpi=600, dx=10, dy=10)   
cm_show('obr.png', width=500)

culling sys module...


In [2]:
#!/usr/bin/env python
'''
Dve elektrody v prostredi
'''
# reset prostredia, pri opakovanom spusteni generuje zle vysledky
%reset -f --aggressive

import gmsh
import numpy
from utils.lib2d import *
import warnings; warnings.filterwarnings("ignore")

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)

                                       # defincia ulohy
R = Rectangle( 0, 0, 10, 10)           # simulacne prostredie
E1 = Circle(-2,0,1)                    # elektroda
E2 = Circle( 2,0,1)                    # elektroda
S = PlaneSurface([R, E1, E2])          # plocha R s dierami C1,C2


                                       # fyzicka reprezentacia  ulohy
PhysicalSurface(S, "Plocha")           # (1) - poradie v Elmer
PhysicalLine(R, "Okraj_priestoru")               # (2)   
PhysicalLine(E1, "Lava elektroda")     # (3)
PhysicalLine(E2, "Prava elektroda")    # (4)


gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)            # 2D mriezka 

gmsh.write("./data/lib2d_01.vtk")
gmsh.write("./data/lib2d_01.msh")
gmsh.write("./data/lib2d_01.geo_unrolled")

gmsh.fltk.run()
gmsh.finalize()


culling sys module...


/usr/lib/python3.13/site-packages/gmsh.py:94: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy
X_ChangeProperty: BadValue (integer parameter out of range for operation) 0x0


In [15]:
'''
Elektrody a dielektrikum medzi nimi
'''
%reset -f --aggressive

import gmsh
import numpy
from utils.lib2d import *

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)

R = Rectangle( 0, 0, 10, 10)           # simulacne prostredie
E1 = Circle(-2,0,1)                    # elektroda
E2 = Circle( 2,0,1)                    # elektroda
D  = RectangleBevel(0,0, .5, 5, .1)    # dielektricke rozhranie

d = PlaneSurface([D])                  # material
s = PlaneSurface([R, E1, E2, D])       # plocha R s dierami C1,C2 a plochou D
                                       # fyzicka reprezentacia  ulohy
PhysicalSurface(s, "Plocha")           # (1) - poradie v Elmer
PhysicalLine(R, "Okraj")               # (2)   
PhysicalLine(E1, "Lava elektroda")     # (3)
PhysicalLine(E2, "Prava elektroda")    # (4)
PhysicalSurface(d, "Dilektrikum")      # (5) 

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("./data/lib2d_02.vtk")
gmsh.write("./data/lib2d_02.msh")
gmsh.write("./data/lib2d_02.geo_unrolled")

gmsh.fltk.run()
gmsh.finalize()

culling sys module...
lines = [1, 2, 3, 4]
lines = [5, 6, 7, 8]
lines = [9, 10, 11, 12]
lines = [13, 14, 15, 16, 17, 18, 19, 20]
loops= [4]
loops= [1, 2, 3, 4]


In [19]:
'''
Vytvaranie objektov pomocou useciek a kriviek.
Zdielanie spolocnych bodov a hranic.
'''
%reset -f --aggressive

import gmsh
import numpy
from utils.lib2d import *
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)

R = Rectangle( 0, 0, 10, 10, "Prostredie")     

P1 = Point(2,0)
P2 = Point(-2,2)
P3 = Point(-2,-2)
Q1 = Polygon([P1, P2, P3], "T1")

P4 = Point(2,4)
Q2 = Polygon([P1, P2, P4], "T2")  # objekt so spolocnymi 

P5 = Point(2, -4)
Q3 = Polygon([P1, P3, P5], "T3")

PlaneSurface([Q2])
PlaneSurface([Q3])
S = PlaneSurface([R,  Q1, Q2, Q3])

PhysicalSurface(S, "Plocha")          

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("./data/lib2d_03.vtk")
gmsh.write("./data/lib2d_03.msh")
gmsh.write("./data/lib2d_03.geo_unrolled")
gmsh.fltk.run()
gmsh.finalize()

culling sys module...
lines = [1, 2, 3, 4]
lines = [5, 6, 7]
lines = [5, 8, 9]
lines = [-7, 10, 11]
loops= [3]
loops= [4]
loops= [1, 2, 3, 4]


In [20]:
%reset -f --aggressive

import gmsh
import numpy
from utils.lib2d import *
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)


R = Rectangle( 0, 0, 10, 10, "Prostredie")    # ohranicene objekty
C1 = Circle(-3, 0, 1)                                 
R1 = RectangleRound(3, 0, 2, 3, .5)

# vytvorenie lomenej ciary z bodov 
L1 = PolyLine([ 
    C1.n,                    # existujuci pociatocny bod
    (0, 1),                  # (x,y) offset k predch. bodu
    (R1.nnw.x - C1.n.x, 0),  # offset k poslednemu zo suradnic bodov 
    R1.nnw                   # absolutna poloha
], 'Horny okraj')

L3 = PolyLine([ R1.ssw, (0,-1.0), (-.5, -.5), (C1.s.x - R1.ssw.x + .5, 0), C1.s], 'Dolny okraj')
LL = Loop( [L1, R1.nw, R1.w, R1.sw, L3, C1.se, C1.ne] )

# vytlacenie vlastnosti objektu
PrintItem(R1)

PlaneSurface([LL])
PlaneSurface([R, LL, C1, R1])          

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("./data/lib2d_04.vtk")
gmsh.write("./data/lib2d_04.msh")
gmsh.write("./data/lib2d_04.geo_unrolled")
gmsh.fltk.run()
gmsh.finalize()

culling sys module...
lines = [1, 2, 3, 4]
lines = [5, 6, 7, 8]
lines = [9, 10, 11, 12, 13, 14, 15, 16]
lines = [17, 18, 19, -16, -15, -14, 20, 21, 22, 23, 8, 5]
                 Meno objektu 
                  Typ objektu 101
            Pozicia stredu x  3
            Pozicia stredu y  0
Zoznam gmsh oznaceni bodov    [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Zoznam gmsh oznaceni useciek  [9, 10, 11, 12, 13, 14, 15, 16]
       Oznacenie gmsh slucky  3

Body
-----+------+-------+-------+-------+----------------------------------------+
 idx | gmsh |   x   |   y   |  name | shared                                 |
-----+------+-------+-------+-------+----------------------------------------+
   0 |   10 |  2.50 |  1.00 |    c1 | 
   1 |   11 |  3.50 |  1.00 |    c2 | 
   2 |   12 |  3.50 | -1.00 |    c3 | 
   3 |   13 |  2.50 | -1.00 |    c4 | 
   4 |   14 |  2.50 |  1.50 |   nnw | 
   5 |   15 |  3.50 |  1.50 |   nne | 
   6 |   16 |  4.00 |  1.00 |   een | 
   7 |   17 |  4.00 